# TRABALHO: DATA QUALITY

## Testes de Qualidade de Dados

### Introdução

Apresentamos neste trabalho um conjunto básico de testes essenciais, focando em ausência de dados (nulls), conformidade de esquema, volume e intervalos de valores. O desafio propõe que os ALUNOS evoluam para testes mais específicos, como validação de valores (finitos, numéricos, de data/hora e de formato), verificação de unicidade e testes de integridade referencial, para obter alertas informativos e contextuais. Finalmente, estes testes poderiam ser incorporados ao Pipeline de Dados para garantia de qualidade.

## ** ATENÇÃO **

Este notebook serve apenas um propósito educativo.

In [19]:
# Em seguida iremos importar diversas bibliotecas que serão utilizadas:

# Pacote para trabalhar com JSON
import json

# Pacote para realizar requisições HTTP
import requests

# Pacote para exploração e análise de dados
import pandas as pd

# Pacote com métodos numéricos e representações matriciais
import numpy as np

# Pacotes do scikit-learn para pré-processamento de dados
# "SimpleImputer" é uma transformação para preencher valores faltantes em conjuntos de dados
from sklearn.impute import SimpleImputer

# Pacotes do scikit-learn para treinamento de modelos e construção de pipelines
# Método para separação de conjunto de dados em amostras de treino e teste
from sklearn.model_selection import train_test_split
# Método para criação de modelos baseados em árvores de decisão
from sklearn.tree import DecisionTreeClassifier
# Classe para a criação de uma pipeline de machine-learning
from sklearn.pipeline import Pipeline

# Pacotes do scikit-learn para avaliação de modelos
# Métodos para validação cruzada do modelo criado
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import accuracy_score

In [20]:
# LE O DATASET COMO UM PANDAS DATAFRAME.
# Certifique-se de que o arquivo 'curso.csv' está no mesmo diretório que este notebook,
# ou forneça o caminho completo para o arquivo.
try:
    df_data_1 = pd.read_csv('curso.txt')
except FileNotFoundError:
    print("Arquivo 'curso.csv' não encontrado. Verifique o caminho e o nome do arquivo.")
    # Você pode querer interromper a execução ou carregar dados de exemplo aqui
    # Por exemplo, criar um DataFrame vazio para evitar erros subsequentes:
    df_data_1 = pd.DataFrame()

if not df_data_1.empty:
    print("Dados originais carregados:")
    print(df_data_1.head())
    print("\nInformações iniciais do DataFrame:")
    df_data_1.info()

Dados originais carregados:
   MATRICULA                       NOME  REPROVACOES_MAT_1  REPROVACOES_MAT_2  \
0     502375          Márcia Illiglener                  0                  0   
1     397093   Jason Jytereoman Izoimum                  0                  0   
2     915288  Bartolomeu Inácio da Gama                  0                  0   
3     192652            Fernanda Guedes                  1                  3   
4     949491     Alessandre Borba Gomes                  1                  3   

   REPROVACOES_MAT_3  REPROVACOES_MAT_4  NOTA_MAT_1  NOTA_MAT_2  NOTA_MAT_3  \
0                  0                  0         6.2         5.8         4.6   
1                  0                  0         6.0         6.2         5.2   
2                  0                  0         7.3         6.7         7.1   
3                  1                  1         0.0         0.0         0.0   
4                  1                  1         0.0         0.0         0.0   

   NOTA_MA

Temos 15 colunas presentes no dataset fornecido, sendo a maioria delas variáveis características (dados de entrada) e uma delas uma variável-alvo (que queremos que o nosso modelo seja capaz de prever). 

As variáveis características são:

    MATRICULA          - número de matrícula do estudante
    NOME               - nome completo do estudante
    REPROVACOES_MAT_1  - número de reprovações na disciplina MAT_1
    REPROVACOES_MAT_2  - número de reprovações na disciplina MAT_2
    REPROVACOES_MAT_3  - número de reprovações na disciplina MAT_3
    REPROVACOES_MAT_4  - número de reprovações na disciplina MAT_4
    NOTA_MAT_1         - média simples das notas do aluno na disciplina MAT_1 (0-10)
    NOTA_MAT_2         - média simples das notas do aluno na disciplina MAT_2 (0-10)
    NOTA_MAT_3         - média simples das notas do aluno na disciplina MAT_3 (0-10)
    NOTA_MAT_4         - média simples das notas do aluno na disciplina MAT_4 (0-10)
    INGLES             - indica se o estudante tem conhecimento em língua inglesa (0 -> não, 1 -> sim, NaN -> não informado)
    H_AULA_PRES        - horas de estudo presencial realizadas pelo estudante
    TAREFAS_ONLINE     - número de tarefas online entregues pelo estudante
    FALTAS             - número de faltas acumuladas do estudante (todas disciplinas)
    
A variável-alvo é:

    PERFIL               - uma *string* que indica uma de cinco possibilidades: 
        "EXCELENTE"      - Estudante não necessita de mentoria
        "MUITO BOM"      - Estudante não necessita de mentoria
        "BOM"            - Estudante não necessita de mentoria
        "REGULAR"        - Estudante necessita de mentoria em algumas matérias
        "DIFICULDADE"    - Estudante necessita de mentoria em várias disciplinas
        
Com um modelo capaz de classificar um estudante em uma dessas categorias, podemos automatizar parte da mentoria estudantil através de assistentes virtuais, que serão capazes de recomendar práticas de estudo e conteúdo personalizado com base nas necessidades de cada aluno.

## TESTES DE QUALIDADE INICIAIS (Após Carga)

# ALUNOS: completar as células abaixo com testes para os dados brutos (df_data_1).

--------------------------------------------------------------------------------
**Testes de Qualidade dos Dados - Etapa Inicial (Após Carga em `df_data_1`)**

**a) TESTES DE SCHEMA**

* **Objetivo:** Verificar se as colunas esperadas estão presentes, com os nomes e tipos de dados corretos. Mudanças de schema são problemas comuns.
* **Instrução para o Aluno:** Verifiquem se as colunas originais do arquivo `curso.csv` foram carregadas corretamente em `df_data_1` e se os tipos de dados inferidos pelo pandas fazem sentido para cada coluna.

In [21]:
print("\nVerificação de Schema em df_data_1:")
if not df_data_1.empty:
    print("Nomes das colunas:", df_data_1.columns.tolist())
    print("Tipos de dados:\n", df_data_1.dtypes)

    # Aluno: Adicione verificações mais específicas, como checar se colunas essenciais existem
    colunas_essenciais = ['MATRICULA', 'NOME', 'REPROVACOES_MAT_1', 'REPROVACOES_MAT_2', 
                           'REPROVACOES_MAT_3', 'REPROVACOES_MAT_4', 'NOTA_MAT_1', 'NOTA_MAT_2', 
                           'NOTA_MAT_3', 'NOTA_MAT_4', 'INGLES', 'H_AULA_PRES', 
                           'TAREFAS_ONLINE', 'FALTAS', 'PERFIL']
    todas_presentes = all(item in df_data_1.columns for item in colunas_essenciais)
    print(f"\nTodas as colunas essenciais estão presentes: {todas_presentes}")
    assert todas_presentes, "Falha: Nem todas as colunas essenciais estão presentes."

    # Aluno: Adicione asserts ou checagens de tipos específicos para colunas importantes.
    # Exemplo:
    # assert pd.api.types.is_integer_dtype(df_data_1['MATRICULA']), "Coluna 'MATRICULA' não é do tipo inteiro como esperado."
    # assert pd.api.types.is_float_dtype(df_data_1['NOTA_MAT_1']), "Coluna 'NOTA_MAT_1' não é do tipo float como esperado."
    # assert pd.api.types.is_object_dtype(df_data_1['PERFIL']), "Coluna 'PERFIL' não é do tipo objeto/string como esperado."
    print("Testes de schema básicos passaram (verifique os asserts descomentados e adaptados)." )
else:
    print("DataFrame df_data_1 está vazio. Testes de schema não podem ser executados.")


Verificação de Schema em df_data_1:
Nomes das colunas: ['MATRICULA', 'NOME', 'REPROVACOES_MAT_1', 'REPROVACOES_MAT_2', 'REPROVACOES_MAT_3', 'REPROVACOES_MAT_4', 'NOTA_MAT_1', 'NOTA_MAT_2', 'NOTA_MAT_3', 'NOTA_MAT_4', 'INGLES', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS', 'PERFIL']
Tipos de dados:
 MATRICULA              int64
NOME                  object
REPROVACOES_MAT_1      int64
REPROVACOES_MAT_2      int64
REPROVACOES_MAT_3      int64
REPROVACOES_MAT_4      int64
NOTA_MAT_1           float64
NOTA_MAT_2           float64
NOTA_MAT_3           float64
NOTA_MAT_4           float64
INGLES               float64
H_AULA_PRES            int64
TAREFAS_ONLINE         int64
FALTAS                 int64
PERFIL                object
dtype: object

Todas as colunas essenciais estão presentes: True
Testes de schema básicos passaram (verifique os asserts descomentados e adaptados).


**b) TESTES DE VOLUME**

* **Objetivo:** Verificar se a quantidade de dados (número de linhas) está dentro do esperado. Importante para monitorar ingestão e custos.
* **Instrução para o Aluno:** Verifiquem o número total de linhas no DataFrame `df_data_1` carregado. Comparem com o número esperado de registros (se souberem) ou apenas verifiquem se não está vazio e se tem um volume mínimo razoável.

In [22]:
print("\nVerificação de Volume em df_data_1:")
if not df_data_1.empty:
    num_linhas = len(df_data_1)
    print(f"Número de linhas no dataset: {num_linhas}")

    # Aluno: Adicione uma verificação se o volume está dentro de um limite aceitável.
    # Suponha que o número mínimo esperado é 50 linhas e o máximo 1000, por exemplo.
    volume_minimo_esperado = 50
    volume_maximo_esperado = 1000 
    assert num_linhas > 0, "Falha: DataFrame está vazio."
    # assert num_linhas >= volume_minimo_esperado, f"Falha: Volume de dados ({num_linhas}) abaixo do esperado (mínimo {volume_minimo_esperado})."
    # assert num_linhas <= volume_maximo_esperado, f"Falha: Volume de dados ({num_linhas}) acima do esperado (máximo {volume_maximo_esperado})."
    print(f"Teste de volume (entre {volume_minimo_esperado} e {volume_maximo_esperado}) passado (verifique os asserts descomentados e adaptados).")
else:
    print("DataFrame df_data_1 está vazio. Testes de volume não podem ser executados.")


Verificação de Volume em df_data_1:
Número de linhas no dataset: 199
Teste de volume (entre 50 e 1000) passado (verifique os asserts descomentados e adaptados).


**c) TESTES DE VALORES (Domínio)**

* **Objetivo:** Verificar se os valores em colunas categóricas ou de domínio restrito pertencem a um conjunto finito de valores esperados. Diferenciar de "corretude".
* **Instrução para o Aluno:** Verifiquem se a coluna `PERFIL` em `df_data_1` contém apenas os valores descritos no enunciado ("EXCELENTE", "MUITO BOM", "BOM", "REGULAR", "DIFICULDADE"). Verifiquem os valores possíveis na coluna `INGLES` antes da transformação (deve ser 0.0, 1.0 ou NaN).

In [23]:
print("\nVerificação de Valores (Domínio) em df_data_1:")
if not df_data_1.empty:
    # Verificação para a coluna 'PERFIL'
    valores_perfil_esperados = sorted(["EXCELENTE", "MUITO BOM", "BOM", "REGULAR", "DIFICULDADE"])
    valores_perfil_encontrados = sorted(df_data_1['PERFIL'].dropna().unique().tolist())
    print(f"Valores únicos esperados em 'PERFIL': {valores_perfil_esperados}")
    print(f"Valores únicos encontrados em 'PERFIL': {valores_perfil_encontrados}")
    
    # Aluno: Adicione código para checar se *todos* os valores encontrados estão nos valores esperados.
    todos_perfis_validos = all(valor in valores_perfil_esperados for valor in valores_perfil_encontrados)
    print(f"Todos os valores de 'PERFIL' encontrados são válidos (estão na lista esperada): {todos_perfis_validos}")
    # assert todos_perfis_validos, "Falha: 'PERFIL' contém valores fora do domínio esperado."

    # Verificação para a coluna 'INGLES' (antes do tratamento de NaN)
    valores_ingles_esperados = sorted([0.0, 1.0]) # NaN é tratado separadamente ou ignorado na checagem de domínio
    valores_ingles_encontrados = sorted(df_data_1['INGLES'].dropna().unique().tolist())
    print(f"\nValores únicos esperados em 'INGLES' (sem NaN): {valores_ingles_esperados}")
    print(f"Valores únicos encontrados em 'INGLES' (sem NaN): {valores_ingles_encontrados}")
    
    # Aluno: Adicione código para checar se *todos* os valores não-NaN encontrados estão nos valores esperados.
    todos_ingles_validos = all(valor in valores_ingles_esperados for valor in valores_ingles_encontrados)
    print(f"Todos os valores de 'INGLES' (sem NaN) são válidos: {todos_ingles_validos}")
    # assert todos_ingles_validos, "Falha: 'INGLES' (sem NaN) contém valores fora do domínio esperado [0.0, 1.0]."
    print("Testes de valores de domínio passaram (verifique os asserts descomentados e adaptados)." )
else:
    print("DataFrame df_data_1 está vazio. Testes de valores não podem ser executados.")


Verificação de Valores (Domínio) em df_data_1:
Valores únicos esperados em 'PERFIL': ['BOM', 'DIFICULDADE', 'EXCELENTE', 'MUITO BOM', 'REGULAR']
Valores únicos encontrados em 'PERFIL': ['BOM', 'DIFICULDADE', 'EXCELENTE', 'MUITO_BOM', 'REGULAR']
Todos os valores de 'PERFIL' encontrados são válidos (estão na lista esperada): False

Valores únicos esperados em 'INGLES' (sem NaN): [0.0, 1.0]
Valores únicos encontrados em 'INGLES' (sem NaN): [0.0, 1.0]
Todos os valores de 'INGLES' (sem NaN) são válidos: True
Testes de valores de domínio passaram (verifique os asserts descomentados e adaptados).


**d) TESTES NUMÉRICOS E DATAS**

* **Objetivo:** Verificar se valores numéricos estão dentro de ranges esperados (mínimo, máximo, etc.) e se datas (se houvesse) estão em formatos e ranges válidos.
* **Instrução para o Aluno:** Verifiquem se as notas (`NOTA_MAT_X`) em `df_data_1` estão dentro de um range razoável (0-10, considerando que antes do tratamento podem ter NaNs). Verifiquem se `REPROVACOES_MAT_X`, `H_AULA_PRES`, `TAREFAS_ONLINE`, `FALTAS` são não-negativos.

In [24]:
print("\nVerificação de Valores Numéricos em df_data_1:")
if not df_data_1.empty:
    # Verificação para as notas (considerando NaN inicialmente)
    notas_cols = ['NOTA_MAT_1', 'NOTA_MAT_2', 'NOTA_MAT_3', 'NOTA_MAT_4']
    print("Checando ranges das notas (0-10):")
    for col in notas_cols:
        min_val = df_data_1[col].min()
        max_val = df_data_1[col].max()
        print(f"  {col}: Min={min_val}, Max={max_val}")
        # Aluno: Adicione asserts ou checagens para verificar se min/max estão dentro de limites razoáveis (e.g., >= 0 e <=10)
        # assert (pd.isna(min_val) or min_val >= 0), f"Falha: Coluna {col} tem valor mínimo ({min_val}) abaixo de 0."
        # assert (pd.isna(max_val) or max_val <= 10), f"Falha: Coluna {col} tem valor máximo ({max_val}) acima de 10."
    print("  Testes de range para notas passaram (verifique os asserts descomentados e adaptados).")

    # Verificação para contagens (reprovações, faltas, etc.) - devem ser não-negativos
    contagem_cols = ['REPROVACOES_MAT_1', 'REPROVACOES_MAT_2', 'REPROVACOES_MAT_3', 'REPROVACOES_MAT_4', 
                       'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS']
    print("\nChecando se contagens são não-negativas:")
    for col in contagem_cols:
         min_val = df_data_1[col].min()
         print(f"  {col}: Min={min_val}")
         # Aluno: Adicione asserts para verificar se o valor mínimo é >= 0
         # assert (pd.isna(min_val) or min_val >= 0), f"Falha: Coluna {col} tem valor mínimo ({min_val}) negativo."
    print("  Testes de não-negatividade para contagens passaram (verifique os asserts descomentados e adaptados).")

    # Aluno: Poderiam adicionar verificações de distribuição (média, mediana, etc.) se relevante.
    # print(f"\nMédia da NOTA_MAT_1: {df_data_1['NOTA_MAT_1'].mean()}")
else:
    print("DataFrame df_data_1 está vazio. Testes numéricos não podem ser executados.")


Verificação de Valores Numéricos em df_data_1:
Checando ranges das notas (0-10):
  NOTA_MAT_1: Min=0.0, Max=8.6
  NOTA_MAT_2: Min=0.0, Max=8.4
  NOTA_MAT_3: Min=0.0, Max=10.0
  NOTA_MAT_4: Min=0.0, Max=8.5
  Testes de range para notas passaram (verifique os asserts descomentados e adaptados).

Checando se contagens são não-negativas:
  REPROVACOES_MAT_1: Min=0
  REPROVACOES_MAT_2: Min=0
  REPROVACOES_MAT_3: Min=0
  REPROVACOES_MAT_4: Min=0
  H_AULA_PRES: Min=0
  TAREFAS_ONLINE: Min=0
  FALTAS: Min=1
  Testes de não-negatividade para contagens passaram (verifique os asserts descomentados e adaptados).


**e) TESTES DE FORMATOS**

* **Objetivo:** Verificar se os valores em colunas de texto ou identificadores seguem um formato ou padrão esperado.
* **Instrução para o Aluno:** Verifiquem se a coluna `MATRICULA` em `df_data_1` contém apenas valores que parecem ser numéricos (o tipo já foi checado no Schema, aqui pode-se pensar em comprimento ou padrão se fosse string). Verifiquem se a coluna `NOME` não está vazia ou contém apenas espaços.

In [25]:
print("\nVerificação de Formatos em df_data_1:")
if not df_data_1.empty:
    # Verificação para 'MATRICULA' - já verificado como integer no schema.
    # Se fosse string, poderíamos checar se todos os caracteres são dígitos:
    # Exemplo: matriculas_sao_digitos = df_data_1['MATRICULA'].astype(str).str.isdigit().all()
    # print(f"Todas as MATRICULAS (convertidas para string) contêm apenas dígitos: {matriculas_sao_digitos}")
    # assert matriculas_sao_digitos, "Falha: Coluna 'MATRICULA' (como string) contém caracteres não-dígitos."
    print("Checando formato da 'MATRICULA' (tipo já verificado no Schema).")

    # Verificação para 'NOME' - não deve estar vazio ou só com espaços
    print("\nChecando formato do 'NOME':")
    nomes_problematicos = df_data_1['NOME'].isnull() | (df_data_1['NOME'].astype(str).str.strip() == '')
    num_nomes_problematicos = nomes_problematicos.sum()
    print(f"  Número de nomes nulos, vazios ou apenas com espaços: {num_nomes_problematicos}")
    # assert num_nomes_problematicos == 0, f"Falha: Encontrados {num_nomes_problematicos} nomes nulos, vazios ou apenas com espaços."
    print("  Teste de nomes não vazios/nulos passou (verifique o assert descomentado e adaptado).")
else:
    print("DataFrame df_data_1 está vazio. Testes de formatos não podem ser executados.")


Verificação de Formatos em df_data_1:
Checando formato da 'MATRICULA' (tipo já verificado no Schema).

Checando formato do 'NOME':
  Número de nomes nulos, vazios ou apenas com espaços: 0
  Teste de nomes não vazios/nulos passou (verifique o assert descomentado e adaptado).


**f) TESTES DE UNICIDADE**

* **Objetivo:** Verificar se colunas que deveriam ter valores únicos (como chaves primárias) de fato não contêm duplicados.
* **Instrução para o Aluno:** Verifiquem se a coluna `MATRICULA` em `df_data_1` contém apenas valores únicos, pois ela provavelmente serve como identificador de cada estudante.

In [26]:
print("\nVerificação de Unicidade em df_data_1:")
if not df_data_1.empty:
    # Verificação para 'MATRICULA' - deve ser única
    matricula_unica = df_data_1['MATRICULA'].is_unique
    print(f"Coluna 'MATRICULA' é única: {matricula_unica}")

    # Aluno: Adicione um assert para falhar o teste se houver duplicados.
    # assert matricula_unica, "Falha: Coluna 'MATRICULA' contém valores duplicados."

    # Opcional: Mostrar duplicados, se houver e se o assert não for usado para parar
    if not matricula_unica:
        duplicados = df_data_1[df_data_1.duplicated(subset=['MATRICULA'], keep=False)]
        print("Linhas com MATRICULA duplicada:")
        print(duplicados)
    print("Teste de unicidade para 'MATRICULA' passou (verifique o assert descomentado e adaptado)." )
else:
    print("DataFrame df_data_1 está vazio. Testes de unicidade não podem ser executados.")


Verificação de Unicidade em df_data_1:
Coluna 'MATRICULA' é única: True
Teste de unicidade para 'MATRICULA' passou (verifique o assert descomentado e adaptado).


**g) TESTES DE INTEGRIDADE REFERENCIAL (Intra-Tabela)**

* **Objetivo:** Verificar relacionamentos lógicos entre colunas dentro da mesma tabela. Em uma única tabela, verifica consistência entre campos relacionados.
* **Instrução para o Aluno:** Como este é um dataset de tabela única, verifiquem a integridade entre colunas relacionadas em `df_data_1`. Por exemplo, se um aluno tem reprovações em uma matéria (`REPROVACOES_MAT_X > 0`), é esperado que a nota correspondente (`NOTA_MAT_X`) seja baixa (por exemplo, menor que 4, conforme a lógica futura do notebook sugere para "REPROVADO"). Ou o contrário, se a nota é alta, não deveria ter reprovações. **Importante:** Este teste pode ser complexo antes das transformações, pois NaN e 0.0 (para quem não cursou) impactam a lógica. Sugiro que os alunos foquem em uma regra simples, como: se `NOTA_MAT_1` é < 4 (e não NaN), `REPROVACOES_MAT_1` deveria ser > 0.

In [27]:
print("\nVerificação de Integridade Referencial (Intra-Tabela) em df_data_1:")
if not df_data_1.empty:
    # Exemplo: Se a nota em MAT_1 é < 4 (e não NaN), deve haver pelo menos 1 reprovação.
    # Esta regra é uma simplificação e pode ter exceções (ex: primeira vez cursando e reprovando).
    condicao_suspeita_mat1 = (df_data_1['NOTA_MAT_1'].notna()) & \
                               (df_data_1['NOTA_MAT_1'] < 4) & \
                               (df_data_1['REPROVACOES_MAT_1'] == 0)
    registros_suspeitos_mat1 = df_data_1[condicao_suspeita_mat1]

    print(f"Número de registros com NOTA_MAT_1 < 4 e 0 reprovações em MAT_1: {len(registros_suspeitos_mat1)}")
    if len(registros_suspeitos_mat1) > 0:
        print("Registros suspeitos (NOTA_MAT_1 < 4 e REPROVACOES_MAT_1 == 0):")
        print(registros_suspeitos_mat1[['NOME', 'NOTA_MAT_1', 'REPROVACOES_MAT_1']])

    # Aluno: Adicione um assert ou uma checagem. Dependendo da regra de negócio real,
    # pode ser que 0 reprovações e nota < 4 seja válido em alguns contextos.
    # Assumindo que isso indica um problema potencial para este exercício:
    # assert len(registros_suspeitos_mat1) == 0, f"Encontrados {len(registros_suspeitos_mat1)} registros suspeitos em MAT_1 (nota baixa sem reprovação registrada)."
    print("Teste de integridade para MAT_1 (nota baixa, 0 reprovações) passou (ou problemas identificados, verifique o assert).")

    # Aluno: Repita ou adapte para outras matérias (MAT_2, MAT_3, MAT_4) ou outras regras de integridade.
    # Exemplo para MAT_2:
    # condicao_suspeita_mat2 = (df_data_1['NOTA_MAT_2'].notna()) & (df_data_1['NOTA_MAT_2'] < 4) & (df_data_1['REPROVACOES_MAT_2'] == 0)
    # registros_suspeitos_mat2 = df_data_1[condicao_suspeita_mat2]
    # print(f"Número de registros com NOTA_MAT_2 < 4 e 0 reprovações em MAT_2: {len(registros_suspeitos_mat2)}")
    # assert len(registros_suspeitos_mat2) == 0, f"Encontrados {len(registros_suspeitos_mat2)} registros suspeitos em MAT_2."

else:
    print("DataFrame df_data_1 está vazio. Testes de integridade referencial não podem ser executados.")


Verificação de Integridade Referencial (Intra-Tabela) em df_data_1:
Número de registros com NOTA_MAT_1 < 4 e 0 reprovações em MAT_1: 0
Teste de integridade para MAT_1 (nota baixa, 0 reprovações) passou (ou problemas identificados, verifique o assert).


--------------------------------------------------------------------------------
### Transformação 1: Tratando dados faltantes para `NOTA_MAT_4` e `INGLES`

Para tratar os dados faltantes em nosso conjunto de dados, iremos agora utilizar uma transformação pronta da biblioteca scikit-learn, chamada **SimpleImputer**, e o método `fillna()` do pandas.

Neste exemplo:
1.  Os valores faltantes em `INGLES` serão preenchidos com `-1` (indicando 'SEM RESPOSTA'). Isso cria `df_data_2`.
2.  Os valores faltantes em `NOTA_MAT_4` serão preenchidos com `0` usando `SimpleImputer`. Isso atualiza `df_data_2` para `df_data_3`.

In [28]:
if not df_data_1.empty:
    print("Valores nulos em df_data_1 ANTES do tratamento de NaN: \n{}".format(df_data_1.isnull().sum()))

    # Etapa 1.1: Tratar INGLES e criar df_data_2
    # df_data_2 será o DataFrame após o preenchimento de NaNs em 'INGLES'
    df_data_2 = df_data_1.copy()
    df_data_2["INGLES"].fillna(-1, inplace=True)
    print("\nValores nulos em df_data_2 (APÓS tratar INGLES com -1): \n{}".format(df_data_2.isnull().sum()))

    # Etapa 1.2: Tratar NOTA_MAT_4 em df_data_2 para criar df_data_3
    # df_data_3 será o DataFrame após a imputação de NaNs em 'NOTA_MAT_4' (e já com 'INGLES' tratado)
    df_data_3 = df_data_2.copy()
    imputer_nota_mat_4 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
    df_data_3[['NOTA_MAT_4']] = imputer_nota_mat_4.fit_transform(df_data_3[['NOTA_MAT_4']])
    # Nota: Se a intenção do notebook original era imputar TODOS os NaNs numéricos com 0,
    # o SimpleImputer deveria ser aplicado de forma mais ampla. Aqui, focamos em NOTA_MAT_4.
    
    print("\n--- Após Tratamento Inicial de Dados Faltantes --- ")
    print("Valores nulos em df_data_3 (APÓS tratar INGLES com -1 e NOTA_MAT_4 com 0): \n{}".format(df_data_3.isnull().sum()))
    # O print original do prompt usava df_data_2.isnull().sum() aqui. 
    # Se df_data_2 é o estado *após* INGLES e NOTA_MAT_4 tratados, então o print deve usar o df final dessa etapa (df_data_3).
    # Se df_data_2 no print do prompt era para mostrar o estado *antes* de NOTA_MAT_4, então o código acima está correto
    # em criar df_data_2 e df_data_3 como estados sequenciais para os testes.
else:
    print("DataFrame df_data_1 está vazio. Transformações de NaN não podem ser executadas.")
    # Define df_data_2 e df_data_3 como vazios para evitar erros nas células de teste
    df_data_2 = pd.DataFrame()
    df_data_3 = pd.DataFrame()

Valores nulos em df_data_1 ANTES do tratamento de NaN: 
MATRICULA             0
NOME                  0
REPROVACOES_MAT_1     0
REPROVACOES_MAT_2     0
REPROVACOES_MAT_3     0
REPROVACOES_MAT_4     0
NOTA_MAT_1            0
NOTA_MAT_2            0
NOTA_MAT_3            0
NOTA_MAT_4           37
INGLES               37
H_AULA_PRES           0
TAREFAS_ONLINE        0
FALTAS                0
PERFIL                0
dtype: int64

Valores nulos em df_data_2 (APÓS tratar INGLES com -1): 
MATRICULA             0
NOME                  0
REPROVACOES_MAT_1     0
REPROVACOES_MAT_2     0
REPROVACOES_MAT_3     0
REPROVACOES_MAT_4     0
NOTA_MAT_1            0
NOTA_MAT_2            0
NOTA_MAT_3            0
NOTA_MAT_4           37
INGLES                0
H_AULA_PRES           0
TAREFAS_ONLINE        0
FALTAS                0
PERFIL                0
dtype: int64

--- Após Tratamento Inicial de Dados Faltantes --- 
Valores nulos em df_data_3 (APÓS tratar INGLES com -1 e NOTA_MAT_4 com 0): 
MATRICULA  

## TESTES DE QUALIDADE (Após Tratamento Inicial de Dados Faltantes)

# ALUNOS: completar as células abaixo com testes para verificar se a transformação de dados faltantes funcionou como esperado.
# Testes para 'INGLES' usarão `df_data_2`.
# Testes para 'NOTA_MAT_4' usarão `df_data_3`.

--------------------------------------------------------------------------------
**Testes de Qualidade dos Dados - Etapa 2 (Após Tratamento de Dados Faltantes)**

**c) TESTES DE VALORES (Revisão)**

* **Objetivo:** Verificar se os valores após o tratamento de NaN estão conforme o esperado (e.g., -1 para `INGLES`).
* **Instrução para o Aluno:** Verifiquem se a coluna `INGLES` em `df_data_2` agora inclui o valor -1.0 e não tem mais NaNs.

In [29]:
print("\nVerificação de Valores ('INGLES') em df_data_2 após fillna(-1):")
if not df_data_2.empty:
    valores_ingles_encontrados_apos = sorted(df_data_2['INGLES'].unique().tolist())
    print(f"Valores únicos em 'INGLES' (df_data_2): {valores_ingles_encontrados_apos}")

    # Aluno: Adicione um assert para verificar se os valores agora são 0.0, 1.0 ou -1.0 e não há NaNs.
    valores_ingles_esperados_apos_tratamento = sorted([-1.0, 0.0, 1.0])
    # assert df_data_2['INGLES'].isin(valores_ingles_esperados_apos_tratamento).all(), "Falha: Coluna 'INGLES' em df_data_2 contém valores inesperados após fillna."
    # assert df_data_2['INGLES'].notna().all(), "Falha: Coluna 'INGLES' em df_data_2 ainda contém valores NaN."
    print("Teste de valores para 'INGLES' em df_data_2 após fillna(-1) passou (verifique os asserts descomentados e adaptados).")
else:
    print("DataFrame df_data_2 está vazio. Testes de valores para INGLES não podem ser executados.")


Verificação de Valores ('INGLES') em df_data_2 após fillna(-1):
Valores únicos em 'INGLES' (df_data_2): [-1.0, 0.0, 1.0]
Teste de valores para 'INGLES' em df_data_2 após fillna(-1) passou (verifique os asserts descomentados e adaptados).


**d) TESTES NUMÉRICOS E DATAS (Revisão)**

* **Objetivo:** Verificar se os ranges ou valores específicos após o tratamento de NaN estão conforme o esperado (e.g., 0 para `NOTA_MAT_4` onde era NaN).
* **Instrução para o Aluno:** Verifiquem se a coluna `NOTA_MAT_4` em `df_data_3` não tem mais NaNs e agora inclui o valor 0 onde antes era NaN. Verifiquem o valor mínimo da coluna `NOTA_MAT_4`.

In [30]:
print("\nVerificação de Valores Numéricos ('NOTA_MAT_4') em df_data_3 após SimpleImputer(fill_value=0):")
if not df_data_3.empty:
    num_nulos_nota4 = df_data_3['NOTA_MAT_4'].isnull().sum()
    print(f"Número de NaNs em 'NOTA_MAT_4' (df_data_3): {num_nulos_nota4}")
    # assert num_nulos_nota4 == 0, f"Falha: Coluna 'NOTA_MAT_4' em df_data_3 ainda contém NaNs: {num_nulos_nota4}."
    print("Teste de ausência de NaN em 'NOTA_MAT_4' (df_data_3) passou (verifique o assert descomentado e adaptado).")

    min_nota4 = df_data_3['NOTA_MAT_4'].min()
    print(f"Valor mínimo em 'NOTA_MAT_4' (df_data_3): {min_nota4}")
    # Aluno: Adicione um assert para verificar se 0 é um dos valores possíveis (ou o mínimo se todos os NaNs se tornaram 0)
    # e se o mínimo não é negativo.
    # assert min_nota4 >= 0, f"Falha: Valor mínimo em 'NOTA_MAT_4' ({min_nota4}) em df_data_3 é negativo."
    # Para checar se 0 está presente se havia NaNs originais:
    # if df_data_1['NOTA_MAT_4'].isnull().any(): # Checa se havia NaNs na original
    #    assert 0.0 in df_data_3['NOTA_MAT_4'].unique(), "Falha: O valor 0.0 (imputado) não está presente em NOTA_MAT_4 em df_data_3, mas deveria se havia NaNs."
    print("Teste de valor mínimo não-negativo e presença de 0 (se aplicável) para 'NOTA_MAT_4' (df_data_3) passou (verifique os asserts descomentados e adaptados).")
else:
    print("DataFrame df_data_3 está vazio. Testes numéricos para NOTA_MAT_4 não podem ser executados.")


Verificação de Valores Numéricos ('NOTA_MAT_4') em df_data_3 após SimpleImputer(fill_value=0):
Número de NaNs em 'NOTA_MAT_4' (df_data_3): 0
Teste de ausência de NaN em 'NOTA_MAT_4' (df_data_3) passou (verifique o assert descomentado e adaptado).
Valor mínimo em 'NOTA_MAT_4' (df_data_3): 0.0
Teste de valor mínimo não-negativo e presença de 0 (se aplicável) para 'NOTA_MAT_4' (df_data_3) passou (verifique os asserts descomentados e adaptados).


Nota-se que não deveriamos ter mais nenhum valor faltante nas colunas `INGLES` e `NOTA_MAT_4` após essas transformações específicas.

### Transformação 2: Criação de novas colunas descritivas (`CURSOU_MAT_X_DESC`) e imputação da média nas notas para casos de 'AINDA NAO CURSOU'

In [31]:
# A entrada para esta transformação é df_data_3
if not df_data_3.empty:
    df = df_data_3.copy() # Usar uma cópia para criar df_data_4

    # 4) Criar colunas CURSOU_MATX_DESC
    # APROVADO = 1, NOTA >= 4 e REPROVACOES == 0
    cond1_mat1 = (df['NOTA_MAT_1'] >= 4) & (df['REPROVACOES_MAT_1'] == 0)
    cond1_mat2 = (df['NOTA_MAT_2'] >= 4) & (df['REPROVACOES_MAT_2'] == 0)
    cond1_mat3 = (df['NOTA_MAT_3'] >= 4) & (df['REPROVACOES_MAT_3'] == 0)
    cond1_mat4 = (df['NOTA_MAT_4'] >= 4) & (df['REPROVACOES_MAT_4'] == 0)
    
    # REPROVADO = 0, NOTA < 4 e REPROVACOES > 0 (ou NOTA < 4 mesmo com REPROVACOES == 0 se for a primeira vez)
    # A lógica original do notebook era: (df['NOTA_MAT_X'] < 4) & (df['REPROVACOES_MAT_X'] > 0)
    # Vamos manter essa lógica. Casos como nota < 4 e reprovações == 0 não seriam classificados por esta regra nem pela de aprovado.
    # Eles seriam 'AINDA NAO CURSOU' se NOTA == 0 e REPROVACOES == 0.
    # Se NOTA_MAT_X é < 4 mas > 0 e REPROVACOES_MAT_X == 0, ficaria sem categoria por estas duas condições.
    # O np.select usará o valor default (0) se nenhuma condição for atendida, ou o último da lista de 'choices'.
    # Para ser mais explícito, a regra de REPROVADO pode ser apenas NOTA < 4 (assumindo que se cursou e não aprovou, é reprovado)
    # No entanto, o notebook original tem uma lógica específica para 'AINDA NAO CURSOU' (NOTA == 0 e REPROVACOES == 0)
    # Vamos seguir a lógica do notebook original para CURSOU_MATX_DESC, que parece ser:
    # 1 (APROVADO), 0 (REPROVADO), -1 (AINDA NAO CURSOU)

    # REPROVADO = 0
    cond2_mat1 = (df['NOTA_MAT_1'] < 4) & (df['REPROVACOES_MAT_1'] > 0) # Lógica original do notebook
    # Uma lógica mais abrangente para reprovado poderia ser apenas (df['NOTA_MAT_1'] < 4) & (df['NOTA_MAT_1'] != 0)
    # Mas isso conflitaria com a definição de 'AINDA NAO CURSOU' se NOTA_MAT_1=0 e REPROVACOES_MAT_1 > 0 (que não deveria ocorrer)
    cond2_mat2 = (df['NOTA_MAT_2'] < 4) & (df['REPROVACOES_MAT_2'] > 0)
    cond2_mat3 = (df['NOTA_MAT_3'] < 4) & (df['REPROVACOES_MAT_3'] > 0)
    cond2_mat4 = (df['NOTA_MAT_4'] < 4) & (df['REPROVACOES_MAT_4'] > 0)

    # AINDA NAO CURSOU = -1 : NOTA = 0, SEM REPROVAÇÕES
    # Esta condição é crucial. NOTA_MAT_4 já foi imputada para 0 se era NaN.
    # Se NOTA_MAT_X == 0 E REPROVACOES_MAT_X == 0, então AINDA NAO CURSOU.
    cond3_mat1 = (df['NOTA_MAT_1'] == 0) & (df['REPROVACOES_MAT_1'] == 0)
    cond3_mat2 = (df['NOTA_MAT_2'] == 0) & (df['REPROVACOES_MAT_2'] == 0)
    cond3_mat3 = (df['NOTA_MAT_3'] == 0) & (df['REPROVACOES_MAT_3'] == 0)
    cond3_mat4 = (df['NOTA_MAT_4'] == 0) & (df['REPROVACOES_MAT_4'] == 0)

    conditions_MAT1 = [cond1_mat1, cond2_mat1, cond3_mat1]
    conditions_MAT2 = [cond1_mat2, cond2_mat2, cond3_mat2]
    conditions_MAT3 = [cond1_mat3, cond2_mat3, cond3_mat3]
    conditions_MAT4 = [cond1_mat4, cond2_mat4, cond3_mat4]

    choices = [1, 0, -1] # 1:APROVADO, 0:REPROVADO, -1:AINDA NAO CURSOU
    # O default do np.select é 0. Se uma linha não se encaixa em nenhuma condição, 
    # CURSOU_MATX_DESC será 0. Isso pode ser problemático se 0 é REPROVADO.
    # É importante que as condições cubram todos os casos ou que o default seja bem pensado.
    # Para este caso, se não for APROVADO, REPROVADO (pela regra estrita), ou AINDA NAO CURSOU,
    # o que seria? Ex: NOTA=3, REPROVACOES=0. Não é APROVADO, não é REPROVADO (pela regra), não é AINDA NAO CURSOU.
    # O ideal seria ter uma categoria 'OUTRO' ou ajustar as regras para serem exaustivas.
    # Por ora, vamos usar um default que não seja uma das categorias válidas, como -99, e depois verificar.
    default_choice = -99 # Um valor para identificar casos não cobertos pelas regras explícitas

    df['CURSOU_MAT1_DESC'] = np.select(conditions_MAT1, choices, default=default_choice)
    df['CURSOU_MAT2_DESC'] = np.select(conditions_MAT2, choices, default=default_choice)
    df['CURSOU_MAT3_DESC'] = np.select(conditions_MAT3, choices, default=default_choice)
    df['CURSOU_MAT4_DESC'] = np.select(conditions_MAT4, choices, default=default_choice)

    # Verificar se algum default foi usado
    for i in range(1,5):
        if default_choice in df[f'CURSOU_MAT{i}_DESC'].unique():
            print(f"ALERTA: Casos não cobertos pelas regras em CURSOU_MAT{i}_DESC. Valor {default_choice} atribuído.")
            # print(df[df[f'CURSOU_MAT{i}_DESC'] == default_choice][['NOME', f'NOTA_MAT_{i}', f'REPROVACOES_MAT_{i}']])
    
    # Imputação da média nas notas para 'AINDA NAO CURSOU'
    MEDIA_NOTA_MAT_1 = df.loc[df['CURSOU_MAT1_DESC'] != -1, 'NOTA_MAT_1'].mean()
    MEDIA_NOTA_MAT_2 = df.loc[df['CURSOU_MAT2_DESC'] != -1, 'NOTA_MAT_2'].mean()
    MEDIA_NOTA_MAT_3 = df.loc[df['CURSOU_MAT3_DESC'] != -1, 'NOTA_MAT_3'].mean()
    MEDIA_NOTA_MAT_4 = df.loc[df['CURSOU_MAT4_DESC'] != -1, 'NOTA_MAT_4'].mean()
    
    # Armazenar médias para referência nos testes
    medias_calculadas = {
        'MEDIA_NOTA_MAT_1': MEDIA_NOTA_MAT_1,
        'MEDIA_NOTA_MAT_2': MEDIA_NOTA_MAT_2,
        'MEDIA_NOTA_MAT_3': MEDIA_NOTA_MAT_3,
        'MEDIA_NOTA_MAT_4': MEDIA_NOTA_MAT_4
    }
    print("\nMédias calculadas (para quem cursou):")
    for key, value in medias_calculadas.items():
        print(f"  {key}: {value:.2f}")

    df.loc[df['CURSOU_MAT1_DESC'] == -1, 'NOTA_MAT_1'] = MEDIA_NOTA_MAT_1
    df.loc[df['CURSOU_MAT2_DESC'] == -1, 'NOTA_MAT_2'] = MEDIA_NOTA_MAT_2
    df.loc[df['CURSOU_MAT3_DESC'] == -1, 'NOTA_MAT_3'] = MEDIA_NOTA_MAT_3
    df.loc[df['CURSOU_MAT4_DESC'] == -1, 'NOTA_MAT_4'] = MEDIA_NOTA_MAT_4

    df_data_4 = df # DataFrame final após esta transformação

    print("\n--- Após Criação de Colunas Descritivas e Imputação de Médias (df_data_4) ---")
    print("Novas colunas adicionadas e médias imputadas.")
    print(df_data_4[['NOME', 'NOTA_MAT_1', 'REPROVACOES_MAT_1', 'CURSOU_MAT1_DESC', 
                     'NOTA_MAT_2', 'REPROVACOES_MAT_2', 'CURSOU_MAT2_DESC']].head())
else:
    print("DataFrame df_data_3 está vazio. Transformação 2 não pode ser executada.")
    df_data_4 = pd.DataFrame() # Define df_data_4 como vazio para evitar erros


Médias calculadas (para quem cursou):
  MEDIA_NOTA_MAT_1: 5.14
  MEDIA_NOTA_MAT_2: 5.00
  MEDIA_NOTA_MAT_3: 4.76
  MEDIA_NOTA_MAT_4: 4.22

--- Após Criação de Colunas Descritivas e Imputação de Médias (df_data_4) ---
Novas colunas adicionadas e médias imputadas.
                        NOME  NOTA_MAT_1  REPROVACOES_MAT_1  CURSOU_MAT1_DESC  \
0          Márcia Illiglener         6.2                  0                 1   
1   Jason Jytereoman Izoimum         6.0                  0                 1   
2  Bartolomeu Inácio da Gama         7.3                  0                 1   
3            Fernanda Guedes         0.0                  1                 0   
4     Alessandre Borba Gomes         0.0                  1                 0   

   NOTA_MAT_2  REPROVACOES_MAT_2  CURSOU_MAT2_DESC  
0         5.8                  0                 1  
1         6.2                  0                 1  
2         6.7                  0                 1  
3         0.0                  3     

## TESTES DE QUALIDADE (Após Criação de Colunas Descritivas e Imputação de Médias)

# ALUNOS: completar as células abaixo com testes para verificar as novas colunas (`CURSOU_MAT_X_DESC`) e os valores de notas imputados em `df_data_4`.

--------------------------------------------------------------------------------
**Testes de Qualidade dos Dados - Etapa Final (Após Criação de Colunas e Imputação de Médias em `df_data_4`)**

**e) TESTES DE VALORES (Novas Colunas)**

* **Objetivo:** Verificar se os valores nas novas colunas criadas (`CURSOU_MAT_X_DESC`) estão conforme o esperado (1, 0, -1, ou o valor default se usado).
* **Instrução para o Aluno:** Verifiquem se as novas colunas `CURSOU_MAT_X_DESC` em `df_data_4` contêm apenas os valores 1, 0 ou -1 (ou o valor default `-99` se as regras não cobriram todos os casos).

In [32]:
print("\nVerificação de Valores (Novas Colunas 'CURSOU_MAT_X_DESC') em df_data_4:")
if not df_data_4.empty:
    cursou_cols = ['CURSOU_MAT1_DESC', 'CURSOU_MAT2_DESC', 'CURSOU_MAT3_DESC', 'CURSOU_MAT4_DESC']
    # Se o default_choice (-99) foi usado e é um valor indesejado, ele deve ser tratado ou as regras ajustadas.
    # Para o teste, vamos considerar os valores esperados como [1, 0, -1] e verificar se -99 aparece.
    valores_cursou_esperados_principais = [1, 0, -1]
    # default_choice_usado_na_transformacao = -99 # definido na célula de transformação

    for col in cursou_cols:
        valores_encontrados = sorted(df_data_4[col].unique().tolist())
        print(f"  Valores únicos em '{col}': {valores_encontrados}")
        
        # Aluno: Adicione assert para verificar se todos os valores encontrados estão nos valores esperados.
        # Primeiro, verifique se o default_choice (-99) está presente, o que indicaria um problema nas regras.
        # assert default_choice_usado_na_transformacao not in valores_encontrados, f"Falha: Coluna '{col}' contém o valor default {default_choice_usado_na_transformacao}, indicando casos não cobertos pelas regras."
        
        # Depois, verifique se os demais valores estão corretos.
        # todos_valores_validos = all(v in valores_cursou_esperados_principais for v in valores_encontrados if v != default_choice_usado_na_transformacao)
        # assert todos_valores_validos, f"Falha: Coluna '{col}' contém valores inesperados além de {valores_cursou_esperados_principais} e {default_choice_usado_na_transformacao}. Encontrados: {valores_encontrados}"
    print("  Testes de valores para 'CURSOU_MAT_X_DESC' passaram (verifique os asserts e a presença do valor default)." )
else:
    print("DataFrame df_data_4 está vazio. Testes de valores para CURSOU_MAT_X_DESC não podem ser executados.")


Verificação de Valores (Novas Colunas 'CURSOU_MAT_X_DESC') em df_data_4:
  Valores únicos em 'CURSOU_MAT1_DESC': [0, 1]
  Valores únicos em 'CURSOU_MAT2_DESC': [0, 1]
  Valores únicos em 'CURSOU_MAT3_DESC': [0, 1]
  Valores únicos em 'CURSOU_MAT4_DESC': [-1, 0, 1]
  Testes de valores para 'CURSOU_MAT_X_DESC' passaram (verifique os asserts e a presença do valor default).


**f) TESTES NUMÉRICOS E DATAS (Após Imputação da Média)**

* **Objetivo:** Verificar se as médias foram calculadas e imputadas corretamente nas notas onde `CURSOU_MAT_X_DESC` é -1.
* **Instrução para o Aluno:** Verifiquem se os registros onde `CURSOU_MAT_1_DESC` é -1 (AINDA NAO CURSOU) em `df_data_4` agora têm o valor da `NOTA_MAT_1` igual à média calculada (`MEDIA_NOTA_MAT_1`). Repitam para as outras matérias.

In [33]:
print("\nVerificação de Valores Numéricos (Após Imputação da Média) em df_data_4:")
if not df_data_4.empty:
    # As médias foram calculadas e armazenadas no dicionário 'medias_calculadas' na célula de transformação.
    # Exemplo para NOTA_MAT_1:
    materia_idx = 1
    col_nota = f'NOTA_MAT_{materia_idx}'
    col_cursou = f'CURSOU_MAT{materia_idx}_DESC'
    media_esperada = medias_calculadas[f'MEDIA_NOTA_MAT_{materia_idx}']
    
    print(f"Verificando {col_nota} para {col_cursou} == -1:")
    print(f"  Média esperada (calculada para quem cursou): {media_esperada:.4f}")

    # Aluno: Verifique os valores de NOTA_MAT_1 onde CURSOU_MAT1_DESC é -1.
    # Use df_data_4 para esta verificação.
    notas_reais_ainda_nao_cursou = df_data_4.loc[df_data_4[col_cursou] == -1, col_nota]
    
    if not notas_reais_ainda_nao_cursou.empty:
        valores_unicos_notas_imputadas = notas_reais_ainda_nao_cursou.unique()
        print(f"  Valores únicos de {col_nota} encontrados para '{col_cursou} == -1': {valores_unicos_notas_imputadas}")
        
        # Aluno: Adicione um assert para verificar se todos os valores encontrados são aproximadamente iguais à média calculada.
        # Pode haver pequena diferença de float, usar np.isclose() ou uma tolerância.
        # for nota_imputada in valores_unicos_notas_imputadas:
        #     assert np.isclose(nota_imputada, media_esperada), f"Falha: Nota imputada {nota_imputada:.4f} para '{col_cursou} == -1' em {col_nota} não é igual à média calculada {media_esperada:.4f}."
        print(f"  Teste de imputação da média para {col_nota} passou (verifique o assert descomentado e adaptado).")
    else:
        print(f"  Não há registros onde {col_cursou} == -1 para {col_nota}, ou a coluna de nota está vazia para esses casos.")

    # Aluno: Repita para as outras matérias (MAT_2, MAT_3, MAT_4) usando um loop ou copiando o bloco.
    # for materia_idx_loop in range(2, 5):
    #     col_nota_loop = f'NOTA_MAT_{materia_idx_loop}'
    #     col_cursou_loop = f'CURSOU_MAT{materia_idx_loop}_DESC'
    #     media_esperada_loop = medias_calculadas[f'MEDIA_NOTA_MAT_{materia_idx_loop}']
    #     print(f"\nVerificando {col_nota_loop} para {col_cursou_loop} == -1:")
    #     print(f"  Média esperada: {media_esperada_loop:.4f}")
    #     notas_reais_loop = df_data_4.loc[df_data_4[col_cursou_loop] == -1, col_nota_loop]
    #     if not notas_reais_loop.empty:
    #         valores_unicos_loop = notas_reais_loop.unique()
    #         print(f"  Valores únicos de {col_nota_loop} encontrados: {valores_unicos_loop}")
    #         for nota_imp_loop in valores_unicos_loop:
    #             assert np.isclose(nota_imp_loop, media_esperada_loop), f"Falha loop: {col_nota_loop}, {nota_imp_loop}, {media_esperada_loop}"
    #     print(f"  Teste de imputação da média para {col_nota_loop} passou.")
else:
    print("DataFrame df_data_4 está vazio. Testes de imputação de média não podem ser executados.")


Verificação de Valores Numéricos (Após Imputação da Média) em df_data_4:
Verificando NOTA_MAT_1 para CURSOU_MAT1_DESC == -1:
  Média esperada (calculada para quem cursou): 5.1367
  Não há registros onde CURSOU_MAT1_DESC == -1 para NOTA_MAT_1, ou a coluna de nota está vazia para esses casos.


**g) TESTES DE INTEGRIDADE REFERENCIAL (Consistência Lógica Após Criação de Colunas)**

* **Objetivo:** Verificar se a lógica das novas colunas `CURSOU_MAT_X_DESC` (1: APROVADO, 0: REPROVADO, -1: AINDA NAO CURSOU) está consistente com os valores originais de `NOTA_MAT_X` e `REPROVACOES_MAT_X` em `df_data_4`, baseados nas condições definidas na transformação.
* **Instrução para o Aluno:** Verifiquem se não há registros onde a coluna `CURSOU_MAT_1_DESC` indica "APROVADO" (1), mas `NOTA_MAT_1` é menor que 4 ou `REPROVACOES_MAT_1` é maior que 0. Verifiquem as outras combinações lógicas definidas para REPROVADO (0) e AINDA NAO CURSOU (-1) para todas as matérias. Cuidado com os casos não cobertos (`default_choice = -99`).

In [34]:
print("\nVerificação de Integridade Referencial (Consistência Lógica das Novas Colunas) em df_data_4:")
if not df_data_4.empty:
    # Exemplo para MAT_1
    materia_idx = 1
    col_nota = f'NOTA_MAT_{materia_idx}'
    col_reprov = f'REPROVACOES_MAT_{materia_idx}'
    col_cursou = f'CURSOU_MAT{materia_idx}_DESC'
    media_imputada_mat1 = medias_calculadas[f'MEDIA_NOTA_MAT_{materia_idx}'] # Usada na regra de AINDA NAO CURSOU

    # Regra 1: Se CURSOU_MAT1_DESC é 1 (APROVADO), então NOTA_MAT_1 >= 4 E REPROVACOES_MAT_1 == 0
    violacoes_mat1_aprovado = df_data_4[
        (df_data_4[col_cursou] == 1) & 
        ((df_data_4[col_nota] < 4) | (df_data_4[col_reprov] > 0))
    ]
    print(f"Número de inconsistências para {col_cursou} (APROVADO): {len(violacoes_mat1_aprovado)}")
    # assert len(violacoes_mat1_aprovado) == 0, f"Falha: Encontradas {len(violacoes_mat1_aprovado)} inconsistências para {col_cursou} (APROVADO)."
    if len(violacoes_mat1_aprovado) > 0: print(violacoes_mat1_aprovado[[col_nota, col_reprov, col_cursou]])

    # Regra 2: Se CURSOU_MAT1_DESC é 0 (REPROVADO), então NOTA_MAT_1 < 4 E REPROVACOES_MAT_1 > 0 (conforme lógica original)
    violacoes_mat1_reprovado = df_data_4[
        (df_data_4[col_cursou] == 0) & 
        ((df_data_4[col_nota] >= 4) | (df_data_4[col_reprov] == 0))
    ]
    print(f"Número de inconsistências para {col_cursou} (REPROVADO): {len(violacoes_mat1_reprovado)}")
    # assert len(violacoes_mat1_reprovado) == 0, f"Falha: Encontradas {len(violacoes_mat1_reprovado)} inconsistências para {col_cursou} (REPROVADO)."
    if len(violacoes_mat1_reprovado) > 0: print(violacoes_mat1_reprovado[[col_nota, col_reprov, col_cursou]])

    # Regra 3: Se CURSOU_MAT1_DESC é -1 (AINDA NAO CURSOU), 
    # então NOTA_MAT_1 (após imputação) é a MÉDIA e REPROVACOES_MAT_1 == 0 originalmente.
    # A checagem da nota média já foi feita. Aqui, verificar REPROVACOES_MAT_1 == 0.
    # E também que a NOTA_MAT_1 original (antes da imputação da média) era 0.
    # Para isso, precisaríamos de df_data_3 (antes da imputação da média para 'AINDA NAO CURSOU')
    # Vamos checar REPROVACOES e a NOTA_MAT_1 imputada.
    violacoes_mat1_nao_cursou_reprov = df_data_4[
        (df_data_4[col_cursou] == -1) & 
        (df_data_4[col_reprov] > 0)
    ]
    print(f"Número de inconsistências para {col_cursou} (AINDA NAO CURSOU - Reprovações > 0): {len(violacoes_mat1_nao_cursou_reprov)}")
    # assert len(violacoes_mat1_nao_cursou_reprov) == 0, f"Falha: Encontradas {len(violacoes_mat1_nao_cursou_reprov)} inconsistências para {col_cursou} (AINDA NAO CURSOU - Reprovações > 0)."
    if len(violacoes_mat1_nao_cursou_reprov) > 0: print(violacoes_mat1_nao_cursou_reprov[[col_nota, col_reprov, col_cursou]])

    # Checar se a nota imputada para -1 é de fato a média (já testado antes, mas pode ser reforçado)
    violacoes_mat1_nao_cursou_nota = df_data_4[
        (df_data_4[col_cursou] == -1) &
        (~np.isclose(df_data_4[col_nota], media_imputada_mat1))
    ]
    print(f"Número de inconsistências para {col_cursou} (AINDA NAO CURSOU - Nota não é a média): {len(violacoes_mat1_nao_cursou_nota)}")
    # assert len(violacoes_mat1_nao_cursou_nota) == 0, f"Falha: {col_cursou} (AINDA NAO CURSOU) com nota diferente da média imputada."
    if len(violacoes_mat1_nao_cursou_nota) > 0: print(violacoes_mat1_nao_cursou_nota[[col_nota, col_reprov, col_cursou]])

    print("Testes de integridade para CURSOU_MAT_X_DESC passaram (verifique os asserts e adapte para outras matérias).")
    # Aluno: Repita essas verificações lógicas para as outras matérias (MAT_2, MAT_3, MAT_4).
else:
    print("DataFrame df_data_4 está vazio. Testes de integridade lógica não podem ser executados.")


Verificação de Integridade Referencial (Consistência Lógica das Novas Colunas) em df_data_4:
Número de inconsistências para CURSOU_MAT1_DESC (APROVADO): 0
Número de inconsistências para CURSOU_MAT1_DESC (REPROVADO): 0
Número de inconsistências para CURSOU_MAT1_DESC (AINDA NAO CURSOU - Reprovações > 0): 0
Número de inconsistências para CURSOU_MAT1_DESC (AINDA NAO CURSOU - Nota não é a média): 0
Testes de integridade para CURSOU_MAT_X_DESC passaram (verifique os asserts e adapte para outras matérias).


--------------------------------------------------------------------------------
### Treinando um modelo de classificação
O restante do notebook com o treino do modelo pode permanecer. É importante que os alunos reflitam sobre como a qualidade dos dados (verificada e melhorada nas etapas anteriores) impacta o desempenho e a confiabilidade do modelo de Machine Learning.

In [35]:
if not df_data_4.empty:
    # Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
    features = [
        #"MATRICULA", # Geralmente não é uma feature para o modelo em si
        "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
        "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
        # As colunas CURSOU_MATX_DESC são representações do estado do aluno, podem ou não ser usadas como features
        # Se usadas, as notas originais para 'AINDA NAO CURSOU' foram alteradas, o que deve ser considerado.
        # Para este exemplo, vamos usar as notas já tratadas e as reprovações.
        # Poderia ser interessante testar o modelo com e sem as colunas CURSOU_MATX_DESC como features.
        'CURSOU_MAT1_DESC', 'CURSOU_MAT2_DESC', 'CURSOU_MAT3_DESC', 'CURSOU_MAT4_DESC'
    ]

    # Definição da variável-alvo
    target = "PERFIL" # Target é uma string, não uma lista de strings para y

    # Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
    X = df_data_4[features]
    y = df_data_4[target]

    # Separação dos dados em um conjunto de treino e um conjunto de teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337, stratify=y) # Adicionado stratify

    # Criação de uma árvore de decisão com a biblioteca ``scikit-learn``:
    decision_tree = DecisionTreeClassifier(random_state=337)

    # Treino do modelo (é chamado o método *fit()* com os conjuntos de treino)
    decision_tree.fit(X_train, y_train)

    # Realização de teste cego no modelo criado
    y_pred = decision_tree.predict(X_test)

    # Acurácia alcançada pela árvore de decisão
    acc = accuracy_score(y_test, y_pred)
    print(f"Acurácia do modelo DecisionTreeClassifier: {100*round(acc, 4)}%")
else:
    print("DataFrame df_data_4 está vazio. Treinamento do modelo não pode ser executado.")

Acurácia do modelo DecisionTreeClassifier: 68.33%


#### SALVA RESULTADO TRANSFORMADO

In [36]:
# Salva em CSV os dados preparados (df_data_4):
if not df_data_4.empty:
    try:
        df_data_4.to_csv("curso_ml_transformado.csv", encoding='utf-8-sig', index=False)
        print("\nDataFrame transformado (df_data_4) salvo em 'curso_ml_transformado.csv'")
    except Exception as e:
        print(f"Erro ao salvar o arquivo CSV: {e}")
else:
    print("DataFrame df_data_4 está vazio. Nada para salvar.")


DataFrame transformado (df_data_4) salvo em 'curso_ml_transformado.csv'


### Gabarito que será usado na correção  (Itens a Conferir no preenchimento dos alunos)

Ao avaliar o trabalho dos alunos, você pode verificar os seguintes pontos em cada seção de testes que eles completarem:

* **Compreensão do Objetivo do Teste:** O aluno demonstrou entender o propósito de cada tipo de teste (Schema, Volume, Valores, etc.)? A descrição ou os comentários no código refletem esse entendimento?
* **Aplicação ao Dataset:** O aluno aplicou o teste corretamente ao dataset `curso.csv` e às transformações específicas realizadas no notebook? Os exemplos escolhidos para testar (e.g., coluna `MATRICULA` para unicidade, `PERFIL` para valores) são relevantes para `df_data_1`, `df_data_2`, `df_data_3` e `df_data_4` conforme a etapa?
* **Correção da Lógica de Teste:** O código Python implementado está correto para realizar a verificação proposta? Ele identifica potenciais problemas nos dados? Foram usados `asserts` de forma eficaz?
* **Utilização de Ferramentas Adequadas:** O aluno utilizou métodos apropriados do pandas ou numpy para realizar as verificações (e.g., `.columns`, `.dtypes`, `len()`, `.unique()`, `.isin()`, `.min()`, `.max()`, `.is_unique`, `.isnull().sum()`, filtragem booleana, `np.isclose()`)?
* **Cobertura:** O aluno tentou aplicar os testes em colunas ou aspectos relevantes dos dados originais e, crucialmente, nos dados transformados e nas novas colunas criadas? (Por exemplo, testar os valores possíveis de `CURSOU_MAT_X_DESC` e a consistência entre `NOTA_MAT_X`, `REPROVACOES_MAT_X` e `CURSOU_MAT_X_DESC` após a transformação).
* **Clareza do Código e Comentários:** O código é legível e bem comentado? Fica claro qual teste está sendo realizado e o que ele verifica?
* **Interpretação dos Resultados:** O aluno comentou sobre os resultados dos testes, especialmente se alguma anomalia ou falha de asserção (se ativada) foi encontrada? Ele consegue explicar o que um resultado inesperado significa?
* **Adaptação dos Exemplos:** O aluno foi além de apenas descomentar os exemplos, adaptando-os ou adicionando novos testes relevantes para as colunas e transformações específicas?
* **Tratamento de Erros/Casos Limites:** O aluno considerou casos como DataFrames vazios ou colunas não existentes ao escrever seus testes (embora o template já ajude com isso)?
* **Reflexão (Opcional):** Em alguns casos (especialmente Integridade Referencial), o aluno identificou nuances ou ambiguidades na regra de negócio ou nos dados brutos (como a questão do valor 0 em `NOTA_MAT_X` que pode significar "não cursou" ou "reprovou com nota zero" antes das transformações explícitas) e ajustou o teste ou comentou sobre isso?